# Notebook 04: Feature Engineering

1. Handle missing values
2. Create lag features for time series prediction
3. Create rolling statistics (mean, std)
4. Prepare target variable for forecasting horizons
5. Split data chronologically (train/val/test/production)
6. Save processed features to S3

### load data

In [23]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
# Add project root to path
sys.path.append("..")

from config.config import (
    BUCKET_NAME, 
    S3_PREFIX, 
    RAW_DATA_FILENAME,
    TARGET_COLUMN,
    SPLIT_RATIOS
)

In [3]:
# Load data from S3
parquet_path = f"s3://{BUCKET_NAME}/data/raw_parquet/{RAW_DATA_FILENAME.replace('.csv', '.parquet')}"
print(f"Loading from: {parquet_path}")

df = pd.read_parquet(parquet_path)

# Convert date
df['date'] = pd.to_datetime(df['date'])

print(f"Shape: {df.shape}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

Loading from: s3://nfci-forecasting-306617143793/data/raw_parquet/state_month_full.parquet
Shape: (12000, 42)
Date range: 2005-01-01 00:00:00 to 2024-12-01 00:00:00


### Handle missing values
- EDA notebook for reference on columns with missing data

In [6]:
# Check missing values before
missing_values = df.isnull().sum()
print("Missing values:")
print(missing_values[missing_values > 0])

Missing values:
PPIFDG        2900
MSPUS         8000
CPI_YOY        600
state_name    3000
dtype: int64


In [9]:
# define function to handle missing values
def handle_missing_values(df):
    """
    Handle missing values based on EDA findings.
    """
    df = df.copy()
    
    # Drop MSPUS (too many missing values - 67%)
    if 'MSPUS' in df.columns:
        df = df.drop(columns=['MSPUS'])
        print("Dropped MSPUS (67% missing)")
    
    # Drop state_name (not needed for modeling, categorical)
    if 'state_name' in df.columns:
        df = df.drop(columns=['state_name'])
        print("Dropped state_name (not needed for modeling)")
    
    # Forward fill PPIFDG within each state
    if 'PPIFDG' in df.columns:
        df['PPIFDG'] = df.groupby('state_fips')['PPIFDG'].ffill()
        # Backfill any remaining at the start
        df['PPIFDG'] = df.groupby('state_fips')['PPIFDG'].bfill()
        print("Forward/back filled PPIFDG")
    
    # Forward fill CPI_YOY within each state
    if 'CPI_YOY' in df.columns:
        df['CPI_YOY'] = df.groupby('state_fips')['CPI_YOY'].ffill()  # forward fill
        df['CPI_YOY'] = df.groupby('state_fips')['CPI_YOY'].bfill() # backfill
        print("Forward/back filled CPI_YOY")
    
    return df

In [10]:

df = handle_missing_values(df)

# Check missing values after handling
missing_after = df.isnull().sum()
print("\nMissing values :")
print(missing_after[missing_after > 0] if missing_after.sum() > 0 else "None!")

Forward/back filled PPIFDG
Forward/back filled CPI_YOY

Missing values :
None!


### Create Lag Features
- features with past values based on time.
- Predictive Features from EDA
    - BAMLH0A0HYM2 (high-yield spread) — strongest correlation
    - UNRATE (unemployment)
    - FEDFUNDS (fed funds rate)
    - SPREAD_10Y_2Y (yield curve)


In [13]:
def create_lag_features(df, columns, lags=[1, 3, 6, 12]):
    """
    Create lag features for specified columns.
    
    Parameters:
        df: DataFrame sorted by date
        columns: List of columns to create lags for
        lags: List of lag periods (in months)
    
    Returns:
        DataFrame with lag features added
    """
    df = df.copy()
    
    # Sort by state and date to ensure correct lag calculation
    df = df.sort_values(['state_fips', 'date']).reset_index(drop=True)
    
    for col in columns:
        if col not in df.columns:
            print(f"  Warning: {col} not found, skipping")
            continue
            
        for lag in lags:
            lag_col_name = f"{col}_LAG{lag}"
            # Shift within each state group
            df[lag_col_name] = df.groupby('state_fips')[col].shift(lag)
        
        print(f"Created lags for {col}: {lags}")
    
    return df

In [14]:
# Columns to create lags for (based on EDA correlations)
lag_columns = [
    'BAMLH0A0HYM2',  # High-yield spread (strongest predictor)
    'UNRATE',        # Unemployment rate
    'FEDFUNDS',      # Fed funds rate
    'SPREAD_10Y_2Y', # Yield curve spread
    'NFCI'           # Target variable
]

# Create lag features (1, 3, 6, 12 months)
df = create_lag_features(df, lag_columns, lags=[1, 3, 6, 12])

print(f"\nShape after lags: {df.shape}")

Created lags for BAMLH0A0HYM2: [1, 3, 6, 12]
Created lags for UNRATE: [1, 3, 6, 12]
Created lags for FEDFUNDS: [1, 3, 6, 12]
Created lags for SPREAD_10Y_2Y: [1, 3, 6, 12]
Created lags for NFCI: [1, 3, 6, 12]

Shape after lags: (12000, 60)


### Create Rolling Windows to capture trends and volatility

In [15]:
def create_rolling_features(df, columns, windows=[3, 6, 12]):
    """
    Create rolling mean and std features.
    
    Parameters:
        df: DataFrame sorted by date
        columns: List of columns to create rolling stats for
        windows: List of window sizes (in months)
    """
    df = df.copy()
    
    # Sort by state and date
    df = df.sort_values(['state_fips', 'date']).reset_index(drop=True)
    
    for col in columns:
        if col not in df.columns:
            print(f"  Warning: {col} not found, skipping")
            continue
            
        for window in windows:
            # Rolling mean (shifted by 1 to avoid data leakage)
            mean_col = f"{col}_RMEAN{window}"
            df[mean_col] = df.groupby('state_fips')[col].transform(
                lambda x: x.shift(1).rolling(window=window, min_periods=1).mean()
            )
            
            # Rolling std
            std_col = f"{col}_RSTD{window}"
            df[std_col] = df.groupby('state_fips')[col].transform(
                lambda x: x.shift(1).rolling(window=window, min_periods=2).std()
            )
        
        print(f"Created rolling stats for {col}: windows={windows}")
    
    return df

In [16]:
# Columns for rolling statistics
rolling_columns = [
    'BAMLH0A0HYM2',
    'UNRATE',
    'NFCI'
]

# Create rolling features (3, 6, 12 month windows)
df = create_rolling_features(df, rolling_columns, windows=[3, 6, 12])

print(f"\nShape after rolling features: {df.shape}")

Created rolling stats for BAMLH0A0HYM2: windows=[3, 6, 12]
Created rolling stats for UNRATE: windows=[3, 6, 12]
Created rolling stats for NFCI: windows=[3, 6, 12]

Shape after rolling features: (12000, 78)


### Create target variable for the forcast horizon

In [17]:
def create_forecast_targets(df, target_col, horizons=[6, 12]):
    """
    Create future target values for different forecast horizons.
    
    Parameters:
        df: DataFrame
        target_col: Name of target column
        horizons: List of forecast horizons (months ahead)
    """
    df = df.copy()
    
    # Sort by state and date
    df = df.sort_values(['state_fips', 'date']).reset_index(drop=True)
    
    for horizon in horizons:
        target_name = f"{target_col}_HORIZON{horizon}"
        # Shift target backwards (future value becomes current row's target)
        df[target_name] = df.groupby('state_fips')[target_col].shift(-horizon)
        print(f"Created {target_name} ({horizon} months ahead)")
    
    return df

In [18]:
# Create forecast targets (6 and 12 months ahead)
df = create_forecast_targets(df, TARGET_COLUMN, horizons=[6, 12])

print(f"\nShape after forecast targets: {df.shape}")

Created NFCI_HORIZON6 (6 months ahead)
Created NFCI_HORIZON12 (12 months ahead)

Shape after forecast targets: (12000, 80)


In [19]:
## view data
df.head()

,state_fips,date,UNRATE,PAYEMS,CIVPART,EMRATIO,U6RATE,AWHMAN,AHETPI,CPIAUCSL,...,UNRATE_RMEAN12,UNRATE_RSTD12,NFCI_RMEAN3,NFCI_RSTD3,NFCI_RMEAN6,NFCI_RSTD6,NFCI_RMEAN12,NFCI_RSTD12,NFCI_HORIZON6,NFCI_HORIZON12
0,1,2005-01-01,5.3,132781,65.8,62.4,9.2,40.7,15.90,191.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.61572,-0.60428
1,1,2005-02-01,5.4,133033,65.9,62.4,9.2,40.7,15.93,192.4,...,5.300,NaN,-0.705360,NaN,-0.705360,NaN,-0.705360,NaN,-0.60043,-0.63435
2,1,2005-03-01,5.2,133152,65.9,62.4,9.1,40.4,15.97,193.1,...,5.350,0.070711,-0.709060,0.005233,-0.709060,0.005233,-0.709060,0.005233,-0.56827,-0.62978
3,1,2005-04-01,5.2,133519,66.1,62.7,9.0,40.4,16.01,193.7,...,5.300,0.100000,-0.695253,0.024198,-0.695253,0.024198,-0.695253,0.024198,-0.58529,-0.62065
4,1,2005-05-01,5.1,133689,66.1,62.8,8.9,40.4,16.03,193.6,...,5.275,0.095743,-0.663980,0.050709,-0.674325,0.046286,-0.674325,0.046286,-0.58353,-0.58318


### Drop NaN rows from lag, rolling and target columns

In [20]:
# Check NaN counts
print(f"Shape before dropping NaN: {df.shape}")
print(f"Total NaN values: {df.isnull().sum().sum()}")

# using 12-month horizon as our primary target
# Drop rows where this target is NaN
primary_target = 'NFCI_HORIZON12'
df_clean = df.dropna(subset=[primary_target]).copy()

print(f"\nShape after dropping rows with NaN target: {df_clean.shape}")

Shape before dropping NaN: (12000, 80)
Total NaN values: 7750

Shape after dropping rows with NaN target: (11400, 80)


In [21]:
# Fill remaining NaN in features (from rolling std with small windows)
# Use 0 for std columns so that no variance if theres not enough data points
std_cols = [c for c in df_clean.columns if 'RSTD' in c]
df_clean[std_cols] = df_clean[std_cols].fillna(0)

# For any remaining NaN, forward fill then backward fill
df_clean = df_clean.groupby('state_fips').apply(
    lambda x: x.ffill().bfill()
).reset_index(drop=True)

# Final check
remaining_nan = df_clean.isnull().sum().sum()
print(f"Remaining NaN values: {remaining_nan}")

if remaining_nan > 0:
    print("Columns with NaN:")
    print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])

Remaining NaN values: 0


### Split data chronologically
- 60% train
- 10% validation
- 10% test
- 20% Production


In [25]:
def chronological_split(df, split_ratios):
    """
    Split data chronologically based on date.
    
    Parameters:
        df: DataFrame with 'date' column
        split_ratios: Dict with 'train', 'validation', 'test', 'production' ratios
    
    Returns:
        Dict with split DataFrames
    """
    # Get unique dates sorted
    dates = sorted(df['date'].unique())
    n_dates = len(dates)
    
    # Calculate split indices
    train_end = int(n_dates * split_ratios['train'])
    val_end = train_end + int(n_dates * split_ratios['validation'])
    test_end = val_end + int(n_dates * split_ratios['test'])
    
    # Get date boundaries
    train_dates = dates[:train_end]
    val_dates = dates[train_end:val_end]
    test_dates = dates[val_end:test_end]
    prod_dates = dates[test_end:]
    
    # Split data
    splits = {
        'train': df[df['date'].isin(train_dates)].copy(),
        'validation': df[df['date'].isin(val_dates)].copy(),
        'test': df[df['date'].isin(test_dates)].copy(),
        'production': df[df['date'].isin(prod_dates)].copy()
    }
    
    # Print summary
    print("Chronological Split Summary:")
    for name, split_df in splits.items():
        date_range = f"{split_df['date'].min().date()} to {split_df['date'].max().date()}"
        print(f"{name:12} | {len(split_df):6,} rows | {date_range}")

    
    return splits

In [ ]:
# Perform the split
splits = chronological_split(df_clean, SPLIT_RATIOS)

Chronological Split Summary:
train        |  6,800 rows | 2005-01-01 to 2016-04-01
validation   |  1,100 rows | 2016-05-01 to 2018-02-01
test         |  1,100 rows | 2018-03-01 to 2019-12-01
production   |  2,400 rows | 2020-01-01 to 2023-12-01


In [28]:
# Verify no data leakage (dates don't overlap)
print("Date ranges verification:")
for name, split_df in splits.items():
    print(f"{name}: {split_df['date'].min().date()} to {split_df['date'].max().date()}")

Date ranges verification:
train: 2005-01-01 to 2016-04-01
validation: 2016-05-01 to 2018-02-01
test: 2018-03-01 to 2019-12-01
production: 2020-01-01 to 2023-12-01


### Define final feature sets and save to s3

In [29]:
# List all feature columns
all_columns = df_clean.columns.tolist()

# Identify column types
id_columns = ['state_fips', 'date']
target_columns = [c for c in all_columns if 'HORIZON' in c] + ['NFCI']
feature_columns = [c for c in all_columns if c not in id_columns + target_columns]

print(f"ID columns ({len(id_columns)}): {id_columns}")
print(f"Target columns ({len(target_columns)}): {target_columns}")
print(f"Feature columns ({len(feature_columns)}): {len(feature_columns)} total")

ID columns (2): ['state_fips', 'date']
Target columns (3): ['NFCI_HORIZON6', 'NFCI_HORIZON12', 'NFCI']
Feature columns (75): 75 total


In [30]:
# Show feature columns grouped
print("\nFeature columns:")

# Original features
original = [c for c in feature_columns if 'LAG' not in c and 'RMEAN' not in c and 'RSTD' not in c]
lag_feats = [c for c in feature_columns if 'LAG' in c]
rolling_feats = [c for c in feature_columns if 'RMEAN' in c or 'RSTD' in c]

print(f"Original features: {len(original)}")
print(f"Lag features: {len(lag_feats)}")
print(f"Rolling features: {len(rolling_feats)}")


Feature columns:
Original features: 37
Lag features: 20
Rolling features: 18


### Save processed data to S3

In [32]:
import boto3
# S3 client
s3_client = boto3.client('s3')

# Save each split to S3
for split_name, split_df in splits.items():
    # Define S3 path
    s3_key = f"{S3_PREFIX[split_name]}/features.parquet"
    s3_uri = f"s3://{BUCKET_NAME}/{s3_key}"
    
    # Save to S3
    split_df.to_parquet(s3_uri, index=False)
    print(f"Saved {split_name}: {s3_uri} ({len(split_df):,} rows)")

Saved train: s3://nfci-forecasting-306617143793/data/splits/train/features.parquet (6,800 rows)
Saved validation: s3://nfci-forecasting-306617143793/data/splits/validation/features.parquet (1,100 rows)
Saved test: s3://nfci-forecasting-306617143793/data/splits/test/features.parquet (1,100 rows)
Saved production: s3://nfci-forecasting-306617143793/data/splits/production/features.parquet (2,400 rows)


### Verify saved data

In [33]:
# List saved files
print("Saved files in S3:")

for split_name in splits.keys():
    prefix = S3_PREFIX[split_name]
    response = s3_client.list_objects_v2(Bucket=BUCKET_NAME, Prefix=prefix)
    
    for obj in response.get('Contents', []):
        size_mb = obj['Size'] / 1024 / 1024
        print(f"  s3://{BUCKET_NAME}/{obj['Key']} ({size_mb:.2f} MB)")

Saved files in S3:
  s3://nfci-forecasting-306617143793/data/splits/train/ (0.00 MB)
  s3://nfci-forecasting-306617143793/data/splits/train/features.parquet (0.14 MB)
  s3://nfci-forecasting-306617143793/data/splits/validation/ (0.00 MB)
  s3://nfci-forecasting-306617143793/data/splits/validation/features.parquet (0.07 MB)
  s3://nfci-forecasting-306617143793/data/splits/test/ (0.00 MB)
  s3://nfci-forecasting-306617143793/data/splits/test/features.parquet (0.06 MB)
  s3://nfci-forecasting-306617143793/data/splits/production/ (0.00 MB)
  s3://nfci-forecasting-306617143793/data/splits/production/features.parquet (0.08 MB)
